In [3]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

c:\Users\divya\OneDrive\Documents\UTAUSTIN\Semester6\460j\460jFinalProject\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

train_path = 'C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester6/460j/460jFinalProject/extracted_features_combined.csv'
# train_path = 'C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester6/460j/460jFinalProject/extracted_features_wav2vec.csv'


In [5]:
data_df=pd.read_csv(train_path, index_col=0)
# data_df=pd.read_csv(train_path)

In [6]:
X = data_df.drop('genre', axis=1).values
y = LabelEncoder().fit_transform(data_df['genre'])

In [7]:
# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


In [8]:
# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = map(
    lambda x: torch.tensor(x, dtype=torch.float32 if x.ndim > 1 else torch.int64),
    (X_train, X_val, y_train, y_val)
)

In [9]:
train_dataset = DataLoader(TensorDataset(X_train, y_train), batch_size=128, shuffle=True)
val_dataset = DataLoader(TensorDataset(X_val, y_val), batch_size=128, shuffle=False)


In [8]:
def define_model(trial):
    # Define model architecture
    n_layers = trial.suggest_int('n_layers', 1, 8)
    layers = []
    in_features = X_train.shape[1]  # Number of input features
    
    # Initialize the number of neurons for the first layer
    out_features = trial.suggest_int('n_units_l0', 64, 10000)  # Set the range for the first layer
    
    for i in range(n_layers):
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        dropout_rate = trial.suggest_float(f'dropout_l{i}', 0.05, 0.5)
        layers.append(nn.Dropout(dropout_rate))
        
        in_features = out_features  # Update in_features for the next layer
        
        if i < n_layers - 1:  # Condition to ensure we don't compute out_features for a non-existent next layer
            # Suggest a reduction factor for the next layer's neuron count
            reduction_factor = trial.suggest_float(f'reduction_factor_l{i}', 1.5, 4)
            out_features = max(int(out_features / reduction_factor), len(np.unique(y)))  # Ensure at least the number of classes

    # Final layer
    layers.append(nn.Linear(in_features, len(np.unique(y))))  # Output layer for classification

    model = nn.Sequential(*layers)
    return model.to(device)


In [9]:
def objective(trial):
    model = define_model(trial)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=trial.suggest_float('lr', 1e-5, 1e-1, log=True))

    # Training and validation loop
    for epoch in range(50):  # This could be a hyperparameter
        model.train()
        for x, labels in train_dataset:
            x, labels = x.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Calculate accuracy
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for x, labels in val_dataset:
                x, labels = x.to(device), labels.to(device)
                outputs = model(x)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        # print(f'Epoch {epoch+1}: Loss = {loss.item()}, Accuracy = {accuracy}%')

    return accuracy  # or return -accuracy if you want to maximize accuracy loss.item()


In [10]:

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)


[I 2024-04-27 18:54:27,759] A new study created in memory with name: no-name-9efcf98e-31ea-40b1-9b95-87105098fb85
[I 2024-04-27 18:54:50,757] Trial 0 finished with value: 50.28142589118199 and parameters: {'n_layers': 5, 'n_units_l0': 6321, 'dropout_l0': 0.38091684238734524, 'reduction_factor_l0': 3.4393866731949387, 'dropout_l1': 0.21004968232038224, 'reduction_factor_l1': 2.6087039437599677, 'dropout_l2': 0.27456339617621, 'reduction_factor_l2': 3.3129609204587087, 'dropout_l3': 0.17282074160271232, 'reduction_factor_l3': 3.8170586286290105, 'dropout_l4': 0.35494202855155066, 'lr': 7.406771463591793e-05}. Best is trial 0 with value: 50.28142589118199.
[I 2024-04-27 18:55:10,211] Trial 1 finished with value: 49.781113195747345 and parameters: {'n_layers': 7, 'n_units_l0': 5158, 'dropout_l0': 0.43508322806095945, 'reduction_factor_l0': 3.2418898909724505, 'dropout_l1': 0.20079364015915696, 'reduction_factor_l1': 1.737316383347237, 'dropout_l2': 0.3941174912355857, 'reduction_factor_l2'

In [1]:
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


Best trial:


NameError: name 'study' is not defined

In [ ]:
!pip install torchsummary

In [2]:
import torch
import torch.nn as nn
from torchviz import make_dot
from torchsummary import summary


In [10]:
def build_best_model(best_params):
    n_layers = best_params['n_layers']
    layers = []
    in_features = X_train.shape[1]  # Assuming X_train is already defined

    for i in range(n_layers):
        out_features = best_params[f'n_units_l{i}']
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        dropout_rate = best_params[f'dropout_l{i}']
        layers.append(nn.Dropout(dropout_rate))
        in_features = out_features

    layers.append(nn.Linear(in_features, len(np.unique(y))))  # Assuming y is your labels array
    model = nn.Sequential(*layers)
    return model


In [12]:
#[I 2024-04-27 20:56:48,444] Trial 338 finished with value: 52.908067542213885 and parameters: {'n_layers': 4, 'n_units_l0': 1304, 'dropout_l0': 0.18235655499375714, 'reduction_factor_l0': 3.610726662142633, 'dropout_l1': 0.36638596505054644, 'reduction_factor_l1': 2.5555945514215552, 'dropout_l2': 0.49104673063428006, 'reduction_factor_l2': 3.8560274115471023, 'dropout_l3': 0.07569054982888096, 'lr': 0.00037268972879675195}. Best is trial 338 with value: 52.908067542213885.
best_params = {'n_layers': 7, 'n_units_l0': 3552, 'dropout_l0': 0.08608922897632487, 'n_units_l1': 2788, 'dropout_l1': 0.49909980517209734, 'n_units_l2': 3943, 'dropout_l2': 0.443142103238337, 'n_units_l3': 2826, 'dropout_l3': 0.4911508678425567, 'n_units_l4': 3285, 'dropout_l4': 0.37983735110144623, 'n_units_l5': 477, 'dropout_l5': 0.2887627489697909, 'n_units_l6': 3122, 'dropout_l6': 0.4197440631958614, 'lr': 0.00022291741485503343}
model = build_best_model(best_params)


In [13]:
# Create a dummy input to visualize the model
dummy_input = torch.zeros(1, X_train.shape[1])

# Visualize the model using torchviz
model_vis = make_dot(model(dummy_input), params=dict(model.named_parameters()))
model_vis.render("neural_network_visualization", format="png", view=True)

# Print model summary using torchsummary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
summary(model, input_size=(X_train.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 3552]       4,212,672
              ReLU-2                 [-1, 3552]               0
           Dropout-3                 [-1, 3552]               0
            Linear-4                 [-1, 2788]       9,905,764
              ReLU-5                 [-1, 2788]               0
           Dropout-6                 [-1, 2788]               0
            Linear-7                 [-1, 3943]      10,997,027
              ReLU-8                 [-1, 3943]               0
           Dropout-9                 [-1, 3943]               0
           Linear-10                 [-1, 2826]      11,145,744
             ReLU-11                 [-1, 2826]               0
          Dropout-12                 [-1, 2826]               0
           Linear-13                 [-1, 3285]       9,286,695
             ReLU-14                 [-

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0002188850943092698)

# Training and validation loop
for epoch in range(50):  # This could be a hyperparameter
    model.train()
    for x, labels in train_dataset:
        x, labels = x.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Calculate accuracy
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, labels in val_dataset:
            x, labels = x.to(device), labels.to(device)
            outputs = model(x)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        for x, labels in train_dataset:
            x, labels = x.to(device), labels.to(device)
            outputs = model(x)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Epoch {epoch+1}: Loss = {loss.item()}, Accuracy = {accuracy}%')

        

Epoch 1: Loss = 1.7532986402511597, Accuracy = 29.647235426569928%
Epoch 2: Loss = 1.5802421569824219, Accuracy = 30.172629472104077%
Epoch 3: Loss = 1.8354936838150024, Accuracy = 34.71353515136352%
Epoch 4: Loss = 1.6793040037155151, Accuracy = 39.87990993244934%
Epoch 5: Loss = 1.7322503328323364, Accuracy = 40.117588191143355%
Epoch 6: Loss = 1.5714045763015747, Accuracy = 39.792344258193644%
Epoch 7: Loss = 1.6061677932739258, Accuracy = 44.62096572429322%
Epoch 8: Loss = 1.474857211112976, Accuracy = 46.622466850137606%
Epoch 9: Loss = 1.4996927976608276, Accuracy = 46.860145108831624%
Epoch 10: Loss = 1.4288493394851685, Accuracy = 47.88591443582687%
Epoch 11: Loss = 1.5116972923278809, Accuracy = 51.06329747310483%
Epoch 12: Loss = 1.458577275276184, Accuracy = 51.613710282712034%
Epoch 13: Loss = 1.3946831226348877, Accuracy = 52.11408556417313%
Epoch 14: Loss = 1.4794516563415527, Accuracy = 51.76382286715036%
Epoch 15: Loss = 1.382555365562439, Accuracy = 54.79109331998999%


In [16]:

# Create a dummy input to visualize the model
dummy_input = torch.zeros(1, X_train.shape[1]).to(device)

# Visualize the model using torchviz
model_vis = make_dot(model(dummy_input), params=dict(model.named_parameters()))
model_vis.render("neural_network_visualization", format="png", view=True)

# Print model summary using torchsummary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
summary(model, input_size=(X_train.shape[1],))

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH